ここに学習で必要なパラメータを定義する

In [2]:
# 表示用
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

In [3]:
import numpy as np
import datetime
import os
import copy
import time

# Parametor about Sensor
SensorNum=1 # Sensor number
#SensorAxis='AccX' # Axis

# Parametor about  " Sliding Window " 
WindowWidth =np.array([128,256,512,1024,2048,4096]) # Window Width
SlidingWidth =WindowWidth/4 #sliding window

# Parametor about Neural Network
AEDimention = 16

# Define Data Name
DATE= str( datetime.date.today() )+'/'
TITLE="Participants01_First_1/"

# Parametor about " Data Path"
DataPath = "/media/takeyama/Transfer/02_ActivityResearch/Paticipants01/01_First/20161018-105301/mem/"
StorePath = "/media/takeyama/Transfer/02_ActivityResearch"

if not os.path.exists(StorePath+'/'+TITLE+DATE+'dictionary/'): 
    os.makedirs(StorePath+'/'+TITLE+DATE+'dictionary/')
DictionaryDataPath=StorePath+'/'+TITLE+DATE+'dictionary/'

if not os.path.exists(StorePath+'/'+TITLE+DATE+'window/'): 
    os.makedirs(StorePath+'/'+TITLE+DATE+'window/')
WindowDataPath=StorePath+'/'+TITLE+DATE+'window/'

if not os.path.exists(StorePath+'/'+TITLE+DATE+'studyOutput/'): 
    os.makedirs(StorePath+'/'+TITLE+DATE+'studyOutput/')
StudyOutputPath=StorePath+'/'+TITLE+DATE+'studyOutput/'

if not os.path.exists(StorePath+'/'+TITLE+DATE+'parametor/'): 
    os.makedirs(StorePath+'/'+TITLE+DATE+'parametor/')
ParametorPath=StorePath+'/'+TITLE+DATE+'parametor/'

if not os.path.exists(StorePath+'/'+TITLE+DATE+'result/'): 
    os.makedirs(StorePath+'/'+TITLE+DATE+'result/')
ResultPath=StorePath+'/'+TITLE+DATE+'result/'


**センサデータの読み込み**

In [4]:
# センサデータのcsvファイル名をリストに集約する関数
def SensorDataFileNameList(path):
    l = os.listdir(DataPath)
    FilePath =[]
    SensorName = []
    
    for n in range(len(l)):
        if (l[n][:4].find('mem-') != -1):
            FilePath.append(l[n])
            SensorName.append(l[n][4:15])
    
    return FilePath,SensorName

CSVを読み込んで，CSVの中身を辞書型で表現する．

In [5]:
def ImportCSV(csv_file,SensorName,mode='Round'):
    import pandas as pd
    '''
    ~Argument~
    csv_file -> ファイル名 
    mode Round -> 四捨五入
         Roundup -> 切り上げ
         Rounddown -> 切り捨て
    
    ~Conversion~
    Acc Data  [0.1mG]=>[G]
    Gyr Data  [0.01dps]=>[dps]   ...dps=degree per second
    '''
    # data dictionary 
    RawData={}   
    AccConversion = 0.1 * 0.001
    GyrConversion = 0.01
    
    # design dataframe and import csv
    data = pd.read_csv(csv_file)
    data.columns=[u'Type',u'Time',u'AccX',u'AccY',u'AccZ',u'GyrX',u'GyrY',u'GyrZ']
    data = data[ data['Type']=='ags']

    # convert numpy.darray 

    AccX=data.AccX.values*AccConversion
    AccY=data.AccY.values*AccConversion
    AccZ=data.AccZ.values*AccConversion
    
    GyrX=data.GyrX.values*GyrConversion
    GyrY=data.GyrY.values*GyrConversion
    GyrZ=data.GyrZ.values*GyrConversion

    # regist each raw data 
    RawData['AccX'] = AccX
    RawData['AccY'] = AccY
    RawData['AccZ'] = AccZ
    RawData['GyrX'] = GyrX
    RawData['GyrY'] = GyrY
    RawData['GyrZ'] = GyrZ
    RawData['Name'] = SensorName

    RawData['Time'] = data.Time.values
    return RawData

In [6]:
import processing
import window

Label = ['LeftHand','RightHand','LeftLeg','Rightleg','West','Chest']

DataFileNameList,SensorName=SensorDataFileNameList(DataPath)
l=[]
for i in range( len(DataFileNameList) ):
    l.append( ImportCSV(DataPath+DataFileNameList[i],Label[i]) )

読み込んだセンサデータの時系列を整理する．
→具体的には，センサデータのスタート時間とゴール時間を整える．


In [7]:
def MakeCommonSection(inputDataArray):
    # 共通区間のスタート時間、ゴール時間を求める
    # スタート時間を求める
    startTime = min(inputDataArray[0]['Time'])
    
    for i in range(len(inputDataArray)):
        if startTime < min(inputDataArray[i]['Time']):
            startTime = min(inputDataArray[i]['Time'])
    
    # ゴール時間を求める
    goalTime = max(inputDataArray[0]['Time'])
    
    for i in range(len(inputDataArray)):
        if goalTime > max(inputDataArray[i]['Time']):
            goalTime = max(inputDataArray[i]['Time'])

    # 共通区間のスタート時間のインデックス、ゴール時間のインデックスを探索する
    def CalcSearchIndexFromTime(data, keyTime):
        """
        data　辞書型
        keyTime data['Time']の中の探す値
        """
        count = 0
        for i in range(0, len(data['Time'])):
            if keyTime == data['Time'][i]:
                print str(keyTime)+' is much in the index  whose number is '+str(i)
                return i
            
    startIndex = np.array([])
    goalIndex = np.array([])
    for obj in inputDataArray:
        print 'start'
        startIndex = np.append(startIndex, CalcSearchIndexFromTime(obj, startTime) ).astype(int)
        print 'goal'
        goalIndex = np.append(goalIndex, CalcSearchIndexFromTime(obj, goalTime) ).astype(int)

    tmp={}
    comDataArray =[]
    key={}

    # センサデータすべて（時刻、加速度、角速度）に対して共通区間のみのデータを抽出
    for number,iDA in enumerate( inputDataArray ):

        tmp['AccX'] = copy.deepcopy( iDA['AccX'][startIndex[number]:goalIndex[number]] )
        tmp['AccY'] = copy.deepcopy( iDA['AccY'][startIndex[number]:goalIndex[number]] )
        tmp['AccZ'] = copy.deepcopy( iDA['AccZ'][startIndex[number]:goalIndex[number]] )
        tmp['GyrX'] = copy.deepcopy( iDA['GyrX'][startIndex[number]:goalIndex[number]] )
        tmp['GyrY'] = copy.deepcopy( iDA['GyrY'][startIndex[number]:goalIndex[number]] )
        tmp['GyrZ'] = copy.deepcopy( iDA['GyrZ'][startIndex[number]:goalIndex[number]] )
        tmp['Time'] = copy.deepcopy( iDA['Time'][startIndex[number]:goalIndex[number]] )
        tmp['Name'] = copy.deepcopy( iDA['Name'] )
        comDataArray.append(copy.deepcopy(tmp) )    
        key[ tmp['Name'] ] = number
       
    return key,comDataArray

In [8]:
TrimKey,TrimData = MakeCommonSection(l)

start
36290413 is much in the index  whose number is 35
goal
37368283 is much in the index  whose number is 1077905
start
36290413 is much in the index  whose number is 0
goal
37368283 is much in the index  whose number is 1077870
start
36290413 is much in the index  whose number is 959
goal
37368283 is much in the index  whose number is 1078829
start
36290413 is much in the index  whose number is 911
goal
37368283 is much in the index  whose number is 1078781
start
36290413 is much in the index  whose number is 871
goal
37368283 is much in the index  whose number is 1078741
start
36290413 is much in the index  whose number is 834
goal
37368283 is much in the index  whose number is 1078704


辞書型に変換したデータをnp.savez関数でnpz型に圧縮し，保存する

In [9]:
for i in range(6):
    np.savez(DictionaryDataPath+TrimData[i]['Name'],data=TrimData[i])

ラベルデータのインポート

In [10]:
TrimData[0]['Time']

array([36290413, 36290414, 36290415, ..., 37368280, 37368281, 37368282])

In [11]:
TrimData[2]['Time']

array([36290413, 36290414, 36290415, ..., 37368280, 37368281, 37368282])

In [12]:
LabelPath = "/media/takeyama/Transfer/02_ActivityResearch/Paticipants01/01_First/20161018-first-syncplay/"
import pandas as pd

In [13]:
data = pd.read_csv(LabelPath+'spa_First_2.csv',encoding='Shift_JIS')

array = pd.to_datetime( data['#Time'].values )
t = np.array([])
for i in tqdm_notebook( range( len(array) ) ):
    t = np.append(t, array[i].hour*3600*1000+array[i].minute*60*1000+array[i].second*1000+array[i].microsecond*0.001)
    
data['#Time'] = t.astype(int)

ラベルとデータのリンクを行う

テストデータとして，Paticipants01のLeftHandのデータにラベルをリンクさせる．

まず，ラベルの時間軸をセンサデータの軸にするために，ラベルのi番目とi+1番目のデータの間に２以上の差があるとき限定でパティングを行う．

2016-11-16
まずは，ラベルデータをセンサデータとリンクするために
* 時間軸のステップが1である
* 時間軸はiとi+1の差が1以上ある場合はその間を差の-1回だけ間にi番目のラベルを挿入する

やっていく処理の流れ
* pandasからnumpyに変換する．
*  

In [14]:
PdArray =data.as_matrix()

In [15]:
print PdArray

[[36295020 u'\u505c\u6b62' nan ..., nan nan nan]
 [36307158 u'\u306a\u3057' u'\u306a\u3057' ..., u'\u306a\u3057'
  u'\u76f4\u7acb' 0.0]
 [36309723 u'\u6301\u3064' u'\u6642\u8a08' ..., u'\u306a\u3057'
  u'\u76f4\u7acb' 101.0]
 ..., 
 [37217223 u'\u306a\u3057' u'\u306a\u3057' ..., u'\u306a\u3057'
  u'\u76f4\u7acb' 114.0]
 [37221638 u'\u306a\u3057' u'\u306a\u3057' ..., u'\u306a\u3057'
  u'\u76f4\u7acb' 114.0]
 [37233827 u'\u306a\u3057' u'\u306a\u3057' ..., u'\u306a\u3057'
  u'\u306a\u3057' 114.0]]


In [16]:
PdArray[0]

array([36295020, u'\u505c\u6b62', nan, nan, nan, nan, nan, nan, nan, nan], dtype=object)

In [ ]:
Output = np.array(PdArray[0])

for i in  tqdm_notebook( range( len(PdArray)-1 ),leave=False ):
    Diff = PdArray[i+1][0]-PdArray[i][0]
    if Diff > 0:
        for l in tqdm_notebook( range(Diff-1),leave=False ):
            AddingArray =PdArray[i]
            AddingArray[0] = AddingArray[0]+(l+1)
            Output = np.vstack( (Output,AddingArray ) )
    Output = np.vstack( (Output, PdArray[i+1]) )
    time.sleep(0.1)

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/takeyama/.pyenv/versions/2.7.11/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/takeyama/.pyenv/versions/2.7.11/envs/takeyama/src/tqdm/tqdm/_tqdm.py", line 102, in run
    for instance in self.tqdm_cls._instances:
  File "/home/takeyama/.pyenv/versions/2.7.11/envs/takeyama/lib/python2.7/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



In [ ]:
np.savez('output',data=Output)

In [ ]:
np.savetxt("MakingLabelArray.csv", Output, delimiter=",")

In [ ]:
def TrimLabel(PandasObject):
    TimeArray = PandasObject['#Time']
    TrimArray = np.array([])
    TrimArray = np.append(TrimArray,PandasObject.ix[0:])

    for i in range( len(TimeArray)-1):
        Diff = TimeArray[i+1]-TimeArray[i]
        if Diff >1:
            for l in range(Diff-1):
                TrimArray = np.append(TrimArray,PandasObject[i:i+1])
            TrimArray = np.append(TrimArray,TimeArray[i+1:i+2])
    return TrimArray

In [ ]:
Output=Output.T

In [ ]:
Output.astype(np.int16)

In [ ]:
index = np.where(Output[0] == 36290413)

In [ ]:
index